In [71]:
import math

class DNode:
    """Class for an internal node of a decision tree"""
    def __init__(self,exs,attributes,possibleVals,targetColName):
        self.__availableAttributes = attributes
        self.__attribute = ''
        self.__examples = exs
        self.__colValues = possibleVals
        self.__targetCol = targetColName
        self.__childNodes = {}
        
        
    def entropy(self,df):
        individEnt = 0.0
        totalEnt = 0.0
        #loop through target column vals
        for x in self.__colValues[self.__targetCol]:
            #where df target col val = the target col val, add onto exList
            exList = df[df[self.__targetCol] == x]
            #print exList
            #find length of exList
            #find length of df
            
            length1 = float(len(exList))
            length2 = float(len(df))
            #print length2
            if length2 == 0.0:
                length2 = 0.00001
            div = length1/length2
            #print div
            
            if div == 0:
                individEnt = 0
            else:
                individEnt = -div*math.log(div,2)
                
            totalEnt += individEnt
        #print totalEnt
        return totalEnt
        
    def chooseAttribute(self): 
        
        minEnt = 100000000.0
        maxInfo = -100000000.0
        
        #self.__attribute = random.choice(self.__availableAttributes) #what a terrible way to choose the attribute!
        
        #first calculate entropy -- say good and vgood are T, acc and unacc are F??
        #get the attribute it is, then find the child nodes, then classify good/vgood as T and acc/unacc as F
        #print self.__examples[self.__targetCol]
        #loop through attributes
        a = ''
        for attr in self.__availableAttributes:
            #print self.__colValues[attr]
            #know possible values, now loop through them
            for individVal in self.__colValues[attr]:
                totalEnt = 0.0
                #find examples that have individual value
                examps = self.__examples[self.__examples[attr] == individVal]
                #print examples[attr]
                #compute the entropy now
                length = len(examps[attr])
                prelimEnt = self.entropy(examps)
                #print examples
                
                #make another function that computes entropy -- done
                length1 = float(len(examps))
                length2 = float(len(self.__examples))
                #print length2
                div = length1/length2
                
                tallyEnt = div*prelimEnt
                
                totalEnt += tallyEnt 
            #find the minimum entropy (inside first for loop)
            if totalEnt < minEnt:
                minEnt = totalEnt
            #find out infogain
            exs = self.__examples
            expectedEntropy = self.entropy(exs)
            infoGain = expectedEntropy - minEnt
            #find maximum infoGain
            if infoGain > maxInfo:
                maxInfo = infoGain
                a = attr
        
        #print infoGain
        self.__attribute = a
      
    def train(self):
        """recursively builds the tree from the training data in __examples"""
        self.chooseAttribute() #'best' attribute at this node

        for v in self.__colValues[self.__attribute]: #going through all possible values this attribute can have
            exsForChild = self.__examples[self.__examples[self.__attribute] == v] #the subset of examples with the given value v
            
            if exsForChild.empty: #if there are no examples to pass to this child,
                                    #make a leaf with the most common among those 
                                    #at this node
                leafChild = DLeaf( self.__examples[self.__targetCol].value_counts().idxmax() )
                self.__childNodes[v] = leafChild #put the leaf in the dictionary of children nodes
                
            elif len(exsForChild[self.__targetCol].unique()) == 1: #all child examples have same class
                leafChild = DLeaf( exsForChild[self.__targetCol].unique()[0] ) #make leaf with that class
                self.__childNodes[v] = leafChild #put the leaf in the dictionary of children nodes
                
            else: #we have a regular decision node for this attribute value
                attributesForChild = list(self.__availableAttributes) #copy attributes
                attributesForChild.remove(self.__attribute) #remove the one we used at this node
                newChild = DNode(exsForChild,attributesForChild,self.__colValues,self.__targetCol)
                newChild.train() #generate the rest of the subtree for this child
                self.__childNodes[v] = newChild #put the new child node in the dictionary of children nodes
            

    def printNode(self,numIndents = 0):
        """display the tree - this isn't the prettiest representation"""
        for i in range(numIndents): print(" "), #print with no newline, this only works for Python 2
        print(self.__attribute)
        for attr in self.__childNodes.keys():
            for i in range(numIndents): print("|"),
            print(":"+attr)
            self.__childNodes[attr].printNode(numIndents+1)
      
    #follows the tree that's already been created
    #be recursive, can be in one line
    def classify(self,testExample):
        #figure out what attribute the node is
        #return that value?
        try:
            childAttr = self.__childNodes[testExample[self.__attribute]]
            return childAttr.classify(testExample)
        except:
            print "error"
                
        
class DLeaf:
    """represents leaves of a decision tree"""
    def __init__(self,labelVal):
        self.__label = labelVal

    def printNode(self,numIndents = 0):
        for i in range(numIndents): print(" "), #print with no newline, this only works for Python 2
        print "LEAF: "+self.__label

    #this method classifies new test data as acc, unacc, good, vgood
    def classify(self,testExample):
        #want to return prediction -- return unacc/acc/good/vgood -- label keeps track of this
        return self.__label    

def binData(colName):
    newCol = list()
    binList = data[colName]
    avg = binList.mean()
    splitFourths = avg / 4
    for x in binList:
        if x < splitFourths:
            newCol.append(0)
        elif x < splitFourths*2:
            newCol.append(1)
        elif x < splitFourths*3:
            newCol.append(2)
        else:
            newCol.append(3)
    return newCol    


        
        
import pandas
import random
from sklearn import cross_validation

#data = pandas.read_csv('carData.csv')
data = pandas.read_csv('german_credit.csv')

#this is a disctionary we'll use to keep track of all
#possible values that any given attribute can have
possibleValuesForEachColumn  = {}

#headers = ['price','maint','doors','persons','trunk','safety','acceptability']
headers = ['Status of existing checking account', 'Duration in month', 'Credit history','Purpose', 'Credit amount', 'Savings account/bonds', 'Present employment since',\
            'Installment rate in percentage of disposable income', 'Personal status and sex', 'Other debtors / guarantors',  'Present residence since',\
             'Property', 'Age in years','Other installment plans ', 'Housing', 'Number of existing credits at this bank', 'Job', 'Number of people being liable to provide maintenance for',\
                 'Telephone', 'foreign worker',  'Creditability']

for h in headers:
    possibleValuesForEachColumn[h] = data[h].unique()
    
cols = ['Credit amount', 'Duration in month', 'Age in years']

for x in cols:
    data[x] = binData(x)
    
#10% of the data goes into the test set, the rest into the training set   
(trainData, testData) = cross_validation.train_test_split(data,test_size = 0.1) 

#initialize the root node
#t = DNode(trainData,['price','maint','doors','persons','trunk','safety'],possibleValuesForEachColumn,'acceptability')
t = DNode(trainData,['Status of existing checking account', 'Duration in month', 'Credit history','Purpose', 'Credit amount', 'Savings account/bonds', 'Present employment since',\
            'Installment rate in percentage of disposable income', 'Personal status and sex', 'Other debtors / guarantors',  'Present residence since',\
             'Property', 'Age in years', 'Other installment plans ','Housing', 'Number of existing credits at this bank', 'Job', 'Number of people being liable to provide maintenance for',\
                 'Telephone', 'foreign worker'],possibleValuesForEachColumn,'Creditability')
t.train() #train the whole tree

#test if classify function works
#t.classify()

#once you add the classify() method to the DNode and DLeaf, you can uncomment the following code

numCorrect = 0.0
for i in range(len(testData)):  #go through all the testing examples
    testExample = testData.iloc[i] #current row
    prediction = t.classify(testExample) #prediction of current row
    if prediction == testExample['Creditability']: #was the prediction correct?
        numCorrect += 1.0

accuracy = numCorrect/len(testData)
print accuracy

#t.printNode() #print the tree to see what it looks like



error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
0.0
